# 송인욱

In [43]:
from bs4 import BeautifulSoup
import platform
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.font_manager as fm
import nltk

In [44]:
if platform.system() == "Windows":
    path = r"c:\Windows\Fonts\malgun.ttf"
    plt.rc('font', family='Malgun Gothic')
        
elif platform.system() == "Darwin":
    path = r"/System/Library/Fonts/AppleGothic"
    plt.rc('font', familly='AppleGothic')

else:
    path = r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

In [45]:
class NaverNews:
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge'

    def __init__(self, search, sort, name=''):
        search = '&query=' + str(search)
        sort = '&sort=' + str(sort)
        self.search = search
        self.sort = sort
        self.name = name
        self.search_url = NaverNews.url + search + sort

    
    def find_tum_text(self, maxpage, tumClass = 'api_txt_lines dsc_txt_wrap'):

        result_list = []

        for num in range(1, maxpage+1):
            page_num = '&start=' + str(10*num-9)
            full_url = self.search_url + page_num 
            req = requests.get(full_url)
            bs = BeautifulSoup(req.text, "html.parser")

            tums = bs.find_all('a', {'class': tumClass})

            for tum in tums:
                result_list.append(tum.text)
        
        return result_list

In [46]:
class DaumWeb:
    url = 'https://search.daum.net/search?nil_suggest=btn&w=fusion&DA=SBC&q='

    def __init__(self, search):
        self.search = search
        self.search_url = DaumWeb.url + search

    def find_tum_text(self, maxpage, tumClass = 'desc'):

        result_list = []

        for i in range(1,maxpage+1):
            full_url = self.search_url + '&p=' + str(i)
            req = requests.get(full_url)
            bs = BeautifulSoup(req.text, "html.parser")


            tums = bs.find_all('p',{'class': tumClass})
            for tum in tums:
                result_list.append(tum.text)

        return result_list

In [48]:
def to_txt(list, filename):
    with open(filename,'w',encoding='UTF-8') as f:
        for txt in list:
            f.write(txt+'\n')

In [49]:
def make_wordcloud(filepath, *delete_sentence):
    f = open(filepath, encoding='utf-8')
    daum_pos = f.read()
    okt = Okt()

    sentences_tag = []
    sentences_tag = okt.pos(daum_pos)  # 형태소 나눔

    noun_adj_list = []
    for word, tag in sentences_tag:
        if tag in ["Noun", "Adjective"]:
            noun_adj_list.append(word)  # 명사 형용사만 추출

    counts = Counter(noun_adj_list)  # 갯수 카운트

    for i in delete_sentence:  # 특정 값 (검색어) 결과에서 삭제
        counts.pop(i)

    tags = counts.most_common(30)

    if platform.system() == "Windows":
        path = r"c:\Windows\Fonts\malgun.ttf"
        plt.rc('font', family='Malgun Gothic')
        
    elif platform.system() == "Darwin":
        path = r"/System/Library/Fonts/AppleGothic"
        plt.rc('font', familly='AppleGothic')
    else:
        path = r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf"


    wc = WordCloud(font_path=path, background_color="white", max_font_size=80)
    cloud = wc.generate_from_frequencies(dict(tags))


    plt.figure(figsize=(10, 8), dpi=100)
    plt.axis("off")
    plt.imshow(cloud, interpolation="bilinear")
    plt.show()

    f.close()


In [50]:
# result = NaverNews('노인+일자리', 0)
# result = result.find_tum_text(100)
# to_txt(result, '네이버_노인+일자리.txt')

In [51]:
# result = DaumWeb('노인+일자리')
# result = result.find_tum_text(100)
# to_txt(result, '다음_노인+일자리.txt')

In [52]:
# make_wordcloud('./네이버_노인+일자리.txt','노인', '일자리')

In [53]:
# make_wordcloud('./다음_노인+일자리.txt', '노인', '일자리')